In [4]:
from sentence_transformers import SentenceTransformer


m = SentenceTransformer('all-MiniLM-L12-v2',)

In [5]:
m.device

device(type='cpu')

In [1]:
from utils.collect_data import keywords

In [2]:
' '.join(keywords)

'faulty oracle punk protocol cosmos call injection internal book-keeping vulcan forged bt finance inconsistent access control akropolis data leak bzx bitcoinica pancakehunny sushi layer-0 opyn flash loan vee finance saddle finance chainswap thorchain growth defi cream finance bridge layer 0 hyperledger indexed finance upbit value defi black hat elephant money alchemix code injection fulcrum furucombo dragonex beanstalk farms paid network pancakebunny soft yearn starstream finance klayswap binance zaif easyfi rollup faulty api polkadot absence of coding logic or sanity check pnetwork gatehub profit calculation error bitgrail interoperability malicious fei protocol venus protocol dao maker bearn fi cashaa bitrue merlin lab xtoken compromised private key-wallet one ring finance security audit direct call to untrusted contract unsafe or infinite token approval bitpoint overledger yapizon interoperability solution autoshark finance memory error coincheck meter token poly network phishing au

In [103]:
mainquery = {
    "query": {
        "bool": {
            "must": {
                "simple_query_string": {
                    "query": ' '.join(f'({k.replace("-","\-")})' for k in keywords),
                    "fields": [
                        "text",
                        "extended_tweet.full_text"
                    ],
                    # 'minimum_should_match': 1
                }
            },
            "filter": [
                {
                    "range": {
                        "created_at": {
                            "gte": "Tue Nov 01 00:00:00 +0000 2022"
                        }
                    }
                },{
                    "bool": {
                        "must_not": {
                            "exists": {
                                "field": "retweeted_status.id"
                            }
                        },
                        "must": {
                            "exists": {
                                "field": "truncated"
                            }
                        }
                    }
                }
            ]
        }
    }
}

In [106]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import json
from utils.collect_data import prettify_elastic

es_options = json.load(open('./twitter_config.json'))['cache_index']

es = Elasticsearch(
    hosts=[es_options['hostname']],
    verify_certs=False,
    # timeout=config.elasticsearch_timeout_secs
)    

cursor = scan(
        es,
        index=es_options['index_name'],
        query = {**mainquery, '_source':es_options['fields']},
        size=10000
    )

In [104]:
doc_count = es.count(
    index=[es_options['index_name']],
    body=mainquery,
    request_timeout=30
)['count']

In [107]:
texts = [
    c['_source']['extended_tweet']['full_text'] if c['_source']['truncated'] else c['_source']['text']
    for c in tqdm(cursor)
]

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [105]:
doc_count

16625047

In [102]:
doc_count

16625047

In [97]:
doc_count

469176

In [85]:
doc_count

22021503

In [ ]:
doc_count

658382

: 

In [2]:
import json

keywords = json.load(open('utils/keywords.json'))

def has_keyword(text):
    return any(
        text.count(k) > 0
        for k in keywords
    )

In [3]:
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
# load the entire snapshot, apply filter, save the filtered ones in diff directory
snapshot_path = Path('/data/blockchain-interoperability/blockchain-social-media/twitter-data_old/snapshots')
# load the entire snapshot, apply filter, save the filtered ones in diff directory
# snapshot_path = Path(snapshot_path)
# huge!
whole_data = pd.concat([
    pd.read_pickle(f)
    for f in tqdm(sorted(snapshot_path.glob('*.pkl')),desc='loading entire dataset..')
])
# good_rows = whole_data[whole_data['whole_text'].apply(has_keyword)]


loading entire dataset..:   0%|          | 0/221 [00:00<?, ?it/s]

In [6]:
whole_data.to_pickle('data.pkl')

In [4]:
from utils.collect_data import has_keyword


whole_data['has_keyword'] = whole_data.whole_text.apply(has_keyword).mean()

KeyboardInterrupt: 

In [1]:
whole_data['has_keyword'].value_counts()

NameError: name 'whole_data' is not defined